# Budgetbuchung Analyse: Datenpräparation

In [ ]:
from pa_lib.data import data_files, load_csv, store_bin, load_bin
from pa_lib.data import desc_col, as_dtype, select_columns, as_date, split_date_iso, make_isoweek_rd, clean_up_categoricals
from pa_lib.util import obj_size
from pa_lib.types import dtFactor

# display long columns completely
pd.set_option('display.max_colwidth', 200)

## Daten einlesen

In [ ]:
data_files()

In [ ]:
pv_bd = load_csv('pv_bd.zip', delimiter=';', encoding='cp1252', dtype='object')

In [ ]:
pv_bd.head()

In [ ]:
display(obj_size(pv_bd), pv_bd.shape)

In [ ]:
desc_col(pv_bd, det=True)

## Leerwerte bereinigen, Datentypen korrigieren

In [ ]:
pv_bd = pv_bd.dropna(how='any')

In [ ]:
(obj_size(pv_bd), pv_bd.shape)

In [ ]:
pv_bd = (pv_bd
         .pipe(as_dtype, 'int', incl_pattern='.*Nr.*')
         .pipe(as_dtype, 'float', incl_pattern='.*tto')
         .pipe(as_date, format='%d.%m.%Y', incl_col=('Res. Datum', 'Aushang Beginn'))
         .pipe(as_dtype, dtFactor, incl_dtype='object'))        

In [ ]:
(obj_size(pv_bd), pv_bd.shape)

In [ ]:
desc_col(pv_bd, det=True)

In [ ]:
pv_bd.head()

## Spaltennamen korrigieren

In [ ]:
pv_bd.columns = ['ResDatum', 'AushangBeginn', 'PvPosNr', 'PartnerNr', 'PartnerName',
       'PvNr', 'PvTitel', 'optBrutto', 'optNetto', 'optNettoNetto']

In [ ]:
pv_bd.head()

## Konstellation Partner/Verträge/Positionen prüfen

#### Gibt es Vertragspositionen mit mehr als einem unterschiedlichen Vertrag? (Wäre nicht toll)

In [ ]:
pv_bd.groupby(['PvPosNr'], observed=True)[['PvNr']].agg('nunique').query('PvNr > 1')

#### Gibt es Verträge mit mehr als einem unterschiedlichen Partner? (Namensänderungen, Handwechsel von Grundstücken, Reorgs...)

In [ ]:
multiPvListe = pv_bd.groupby('PvNr')[['PartnerNr']].agg('nunique').query('PartnerNr > 1').index.values

def partners(x):
    return pd.Series({'#partners': x['PartnerNr'].nunique(), 'pNr': set(x['PartnerNr']), 'pNames': ' | '.join(set(x['PartnerName']))})

(pv_bd.loc[pv_bd.PvNr.isin(multiPvListe)]
      .groupby('PvNr').apply(partners)
      .sort_values(['#partners', 'pNames'], ascending=[False, True]))

## Sortieren, Geschäftsjahr und -woche für Aushang und Reservation berechnen

In [ ]:
pv_bd = (pv_bd.sort_values('AushangBeginn').reset_index(drop=True)
         .pipe(split_date_iso, dt_col='ResDatum', yr_col='RJahr', kw_col='RKw')
         .pipe(split_date_iso, dt_col='AushangBeginn', yr_col='AJahr', kw_col='AKw')
         .reset_index(drop=True))

In [ ]:
desc_col(pv_bd, det=True)

## Daten speichern

In [ ]:
store_bin(pv_bd, 'pv_bd_raw.feather')